<a href="https://colab.research.google.com/github/mojtabaSefidi/Spam-Message-Recognition/blob/main/Spam_Message_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam Recognition

The SMS Spam Collection is a set of SMS tagged messages that have been collected for
SMS Spam research. It contains one set of SMS messages in English of 5,574 messages,
tagged acording being ham (legitimate) or spam.

<b> purpose : Design Model for recognize Spam messages </b>

## Import the relevant libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from string import punctuation
sn.set()

## Load the Dataset

In [4]:
!gdown --id 1gU5LpNVzIesIDPP8vHVqDKRX0btb9fNb
sms_dataset = pd.read_csv("Spam_Sms_DataSet.txt", sep='\t')
sms_dataset.head()

Downloading...
From: https://drive.google.com/uc?id=1gU5LpNVzIesIDPP8vHVqDKRX0btb9fNb
To: /content/Spam_Sms_DataSet.txt
100% 478k/478k [00:00<00:00, 31.4MB/s]


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
